### Import libraries

In [2]:
# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os

%matplotlib inline

### Define functions

In [3]:
def take_tail_fromcigar(strand, cig, seq):
    '''
    Function returns tail seqence based on the CIGAR and sequence of read
    Input:
        cig: CIGAR from sam file after alignment qith soft-clipping  [str]
        seq: sequence of read started from 3' end, having posibble tail   [str]
    Output:
        sequence of the tail extracted from read sequence based on CIGAR [str]
    Assumptions:
        1. For tailed reads, CIGAR starts from \d+S (softcliping code). It's about the number
        of softclipped nucleotides. We assume that this is also the length of the polyA tail.
        2. Sequence seq starts from 3' tail of read.
    '''

    cigar2 = str(cig)
    if not "S" in cigar2:
        return ""
    else: 
        
        
        if strand =='-':      
            search_result_minus = re.findall("[0-9]+S$", cigar2)
            if search_result_minus: 
                number_S = int(search_result_minus[-1].split('S')[0])
               
                tail = seq[-number_S:]
                return tail
            else:
                return ""
        elif strand == '+':
            search_result_plus = re.findall("^[0-9]+S", cigar2)
            if search_result_plus:
                number_S = int(cigar2.split('S')[0])
                tail = seq[0:number_S] 
                return tail
            else:
                return ""
        else:
             return ""
        

##### Test the function take_tail_fromcigar() #####

assert take_tail_fromcigar('-', '8S', 'UUUHSAAAAAAAAAA') =='AAAAAAAA'
assert take_tail_fromcigar('-', '8S444M1S', 'AAAAAAAAAAUUUHS') =='S'
assert take_tail_fromcigar('+', '8S444M3S', 'AAAAAAAAAAUUUHS') =='AAAAAAAA'
assert take_tail_fromcigar('-', '67M7S', 'AAAAAAAAAAUUUHS') =='AAUUUHS'
assert take_tail_fromcigar('-', '*', 'AAAAAAAAAAUUUHS') ==''

In [64]:
def take_tail_fromcigar_revstrand(strand, cig, seq):
    '''
    Function returns tail seqence based on the CIGAR and sequence of read
    Input:
        cig: CIGAR from sam file after alignment qith soft-clipping  [str]
        seq: sequence of read started from 3' end, having posibble tail   [str]
    Output:
        sequence of the tail extracted from read sequence based on CIGAR [str]
    Assumptions:
        1. For tailed reads, CIGAR starts from \d+S (softcliping code). It's about the number
        of softclipped nucleotides. We assume that this is also the length of the polyA tail.
        2. Sequence seq starts from 3' tail of read.
    '''

    cigar2 = str(cig)
    if not "S" in cigar2:
        return ""
    else: 
        
        
        if strand =='+':      
            search_result_minus = re.findall("[0-9]+S$", cigar2)
            if search_result_minus: 
                number_S = int(search_result_minus[-1].split('S')[0])
               
                tail = seq[-number_S:]
                return tail
            else:
                return ""
        elif strand == '-':
            search_result_plus = re.findall("^[0-9]+S", cigar2)
            if search_result_plus:
                number_S = int(cigar2.split('S')[0])
                tail = seq[0:number_S] 
                return tail
            else:
                return ""
        else:
             return ""
        

In [4]:
pattern_polyAU_forward = re.compile("^A{1,}T{1,}") # reverse transcribed
pattern_oligoU_forward = re.compile("^A{1,}")
pattern_polyA_forward = re.compile("^T{1,}$")

pattern_polyAU_reverse = re.compile("A{1,}T{1,}$") # reverse transcribed
pattern_oligoU_reverse = re.compile("T{1,}$")
pattern_polyA_reverse = re.compile("A{1,}$")  

def test_tail(strand, tail): ## poprawiona wersja january 2023
    """ Tests for the type of tail. Patterns are prepared for analysis of R2 read, reverse transcribed:
        base sequence start from the end of polyadenylation tail at 3' end of RNA,
        where T means adenine, and A meand uridine in the 3' tail"""
    if len(tail) > 0:
        if ("G" in tail or "C" in tail):
            return "mix_GC"
        else:
            
            if strand == '-' or strand == 'unmapped': ## podmianka +*-

                if "A" in tail:
                    if pattern_polyAU_forward.fullmatch(tail): 
                        return "polyAU"
                    elif pattern_oligoU_forward.fullmatch(tail): 
                        return "oligoU"
                    else:
                        return "mixed_tail"
                elif pattern_polyA_forward.fullmatch(tail):
                        return "polyA"
                else: 
                    return "no_asigned_tail"
            elif strand == '+': ## podmianka +*-
                if "T" in tail:
                    if pattern_polyAU_reverse.fullmatch(tail): 
                        return "polyAU"
                    elif pattern_oligoU_reverse.fullmatch(tail): 
                        return "oligoU"
                    else:
                        return "mixed_tail"
                elif pattern_polyA_reverse.fullmatch(tail):
                        return "polyA"
                else: 
                    return "no_asigned_tail"
            else:
                return "else_tail"

In [5]:
pattern_mRNA = re.compile("spac|spap|spbp|spcc|spcp|spbc")
pattern_tRNA = re.compile("trna")
pattern_mito = re.compile("spmit")
pattern_rRNA = re.compile("rrna")
pattern_snRNA = re.compile("snrna")
pattern_snoRNA = re.compile("snorna")
pattern_ncRNA = re.compile("ncrna")


def test_RNA_type(feature_name):
    """tests for the type of tail """
    if pattern_mRNA.search(feature_name): # for example AATT
        return "mRNA"
    elif pattern_tRNA.search(feature_name): # for example AATT
        return "tRNA"
    elif pattern_mito.search(feature_name): # for example AATT
        return "mito_RNA"
    elif pattern_rRNA.search(feature_name): # for example AATT
        return "rRNA"
    elif pattern_snRNA.search(feature_name): # for example AATT
        return "snRNA"
    elif pattern_snoRNA.search(feature_name): # for example AATT
        return "snoRNA"
    elif pattern_ncRNA.search(feature_name): # for example AATT
        return "ncRNA"
    else: #only A's
        return "other_type"

assert test_RNA_type("spncrna") == "ncRNA"
assert test_RNA_type("sspcc") == "mRNA"
assert test_RNA_type("AAAAA") == "other_type"
assert test_RNA_type("TAATTT") == "other_type"

In [69]:
def detect_tails_from_CIGAR_new(bam_file_path, references_bed_path,
                            output_path='output_tailsfromcigar.csv'):
    #sam_file = pd.read_csv(sam_file_path, index=False)
    #sam_file.to_csv('sam_input.sam',index=False)
   # references_file = pd.read_csv(references_bed_path)
    #references_file.to_csv('references.bed', sep ='\t', indexl=False)
    


    os.system('samtools view -F 256 {} > primaalign.sam'.format(bam_file_path))
    #os.system('cp {}_primaalign.sam temporary.sam'.format(bam_file_path))

    os.system("awk -F'\t' '$6~/S/'  primaalign.sam > temporary_awk.sam")

    os.system("sam2bed temporary_awk.sam temporary_awk.bed") 

    os.system("bedtools sort -i temporary_awk.bed > temporary_awksorted.bed") 


    os.system("bedtools closest -s -d -t first -a temporary_awksorted.bed -b {} >  closest.bed".format(references_bed_path))  

    os.system("awk '{print  $1, $2, $3, $4, $5, $6,  $8, $9, $10, $17  }' closest.bed > closest_lesscol.bed")

    os.system("awk '{ print $1,$6, $10 }' temporary_awk.sam > primaalign_awk_col6_10.csv")
   
    df_closest = pd.read_csv('closest_lesscol.bed', sep = ' ',
                             names = ['chr',  'start', 'stop', 'read_id', 'num0', 'strand', 'start_gene',
                                      'stop_gene', 'gene_id',   'distance_clos'])
    df_minisam =  pd.read_csv('primaalign_awk_col6_10.csv', sep = ' ', 
                     names = [ 'read_id', 'cigar', 'seq'])
    cigar = pd.merge(df_closest , df_minisam, on = 'read_id')
    cigar['gene'] = cigar['gene_id'].apply(lambda x: x.split(':')[0])
    
    cigar['tail_fromcigar'] = cigar.apply(lambda kol: take_tail_fromcigar_revstrand(kol.strand, 
                                                                          kol.cigar,kol.seq), axis = 1)
    cigar['tail_type'] = cigar.apply(lambda kol: test_tail(kol.strand, kol.tail_fromcigar), axis = 1)
    cigar['tail_len'] = cigar['tail_fromcigar'].apply(lambda x: len(str(x)))
    cigar['id_lower'] = cigar['gene_id'].apply(lambda x: x.lower())
    cigar['RNA_type'] = cigar['id_lower'].apply(lambda x: test_RNA_type(x))
    cigar.to_csv(output_path, index = False)
   # return cigar

In [70]:
### TEST the code
detect_tails_from_CIGAR_new(bam_file_path = '../mapped/WT_2.bam', 
                            references_bed_path = '../reference/newSP_gene.bed',
                            output_path='output_tailsfromcigar.csv')